In [2]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
from scipy import signal
import random

import h5py
from IPython.utils import io

import numpy as np
import matplotlib.pyplot as plt
import matplotlib
from scipy import signal
import random
import scipy.stats as stats

import h5py
from IPython.utils import io

from joblib import Parallel, delayed
from tqdm import tqdm
import pandas as pd

In [3]:
import sys
sys.path.append('../../../../nadavp/new_trap/Tools/')
sys.path.append('../../../../nadavp/new_trap/Tools/StatFramework/')
import BeadDataFile
from discharge_tools import *

from likelihood_calculator import likelihood_analyser
from likelihood_calculator import GravityFramework
from likelihood_calculator import auxiliary_functions as aux
gfw = GravityFramework.GravityFramework()

sys.path.append('/home/analysis_user/New_trap_code/SensitivityFramework/')
from signal_model_utilities_v3b import *

In [4]:
%matplotlib widget

In [5]:
%reload_ext autoreload
%autoreload 2

### Setup parameters

In [6]:
df = pd.read_pickle("/data/new_trap_processed/processed_files/20200320/Bead1/Shaking/Shaking378/Shaking3_aux.pkl") #here one could put the respective file name
height_cal = df["z_distance_emp"]
time = df['Time_Epoch']
time -= time[0]

In [7]:
fname = r'/home/data_transfer/data_from_windows/20200320/Bead1/TransFunc/Repeat_TransFunc_20200104/TransFunc_X_m300k_250s_1hz.h5'
tf_x = BeadDataFile.BeadDataFile(fname)
fname = r'/home/data_transfer/data_from_windows/20200320/Bead1/TransFunc/Repeat_TransFunc_20200104/TransFunc_Y_m300k_250s_1hz.h5'
tf_y = BeadDataFile.BeadDataFile(fname)
fname = r'/home/data_transfer/data_from_windows/20200320/Bead1/TransFunc/Repeat_TransFunc_20200104/TransFunc_Z_m300k_250s_1hz.h5'
tf_z = BeadDataFile.BeadDataFile(fname)
gfw.build_transfer_function([tf_x, tf_y, tf_z], base_freq=1, number_of_harmonics=90, scale_freq=71, plot=False)

No height data
No cantilever data
Error loading spinning data
No laser power data
No height data
No cantilever data
Error loading spinning data
No laser power data
No height data
No cantilever data
Error loading spinning data
No laser power data


In [8]:
bdf_discharge = load_dir(dirname='/data/new_trap/20200320/Bead1/Discharge/Discharge_after_Mass_20200402/After_spinning_again_20200403/',
                         start_file=230, max_file=40)
bdf_z = bdf_discharge[5:35]

530  files in folder
40  files loaded


In [9]:
bandwidth = 2
m1_zscale = gfw.build_z_response(bdf_z, drive_freq=71, charges=6, bandwidth=bandwidth, decimate=10)

***************************************************
Z2-amplitude:  2.70e+02
reduced chi2:  175.24060921459508
***************************************************
Z2-amplitude:  2.61e+02
reduced chi2:  392.683293718069
***************************************************
Z2-amplitude:  2.75e+02
reduced chi2:  233.0708308046279
***************************************************
Z2-amplitude:  2.79e+02
reduced chi2:  341.81841601739336
***************************************************
Z2-amplitude:  2.81e+02
reduced chi2:  320.1151243975466
***************************************************
Z2-amplitude:  2.75e+02
reduced chi2:  235.238026011364
***************************************************
Z2-amplitude:  2.70e+02
reduced chi2:  250.1490478568309
***************************************************
Z2-amplitude:  2.66e+02
reduced chi2:  270.7275591713835
***************************************************
Z2-amplitude:  2.67e+02
reduced chi2:  318.6342737392779
*******************

In [10]:
print("check if scale is positive: ", gfw.scale_Z2)

check if scale is positive:  -1.84743267861e+17


In [9]:
bdf_discharge = load_dir(dirname='/data/new_trap/20200320/Bead1/Discharge/New_Discharge/',
                         start_file=0, max_file=1000)
bdf_x = bdf_discharge[5:10]
bdf_y = bdf_discharge[10:]

No laser power data
No laser power data
No laser power data
No laser power data
No laser power data
No laser power data
No laser power data
No laser power data
No laser power data
No laser power data
No laser power data
No laser power data
No laser power data
No laser power data
No laser power data
15  files in folder
15  files loaded


In [10]:
bandwidth = 2
m1_xscale = gfw.build_x_response(bdf_x, drive_freq=71, charges=7, bandwidth=bandwidth, decimate=10)

***************************************************
X2-amplitude:  6.05e+00
reduced chi2:  0.09133757316259246
***************************************************
X2-amplitude:  6.17e+00
reduced chi2:  0.12856232450832789
***************************************************
X2-amplitude:  6.14e+00
reduced chi2:  0.13973800608162576
***************************************************
X2-amplitude:  6.00e+00
reduced chi2:  0.08988917812669067
***************************************************
X2-amplitude:  6.20e+00
reduced chi2:  0.11019511470165794
X3 to X2 ratio: 0.0835409151805
X2 response (amplitude): 6.11458670456
X2 response (amplitude): 6.11458670456


In [11]:
def get_Yoffset(i):
    fname = r'/data/new_trap/20200320/Bead1/Shaking/Shaking378/Shaking3_'+str(i)+'.h5'
    bb = BeadDataFile.BeadDataFile(fname=fname)    
    return (437.3 - 25 * 9.5) - (aux.voltage_to_position(np.mean(bb.cant_pos[1])))
def get_Stroke(i):
    fname = r'/data/new_trap/20200320/Bead1/Shaking/Shaking378/Shaking3_'+str(i)+'.h5'
    bb = BeadDataFile.BeadDataFile(fname=fname)    
    return -np.std(aux.voltage_to_position(bb.cant_pos[1]))*np.sqrt(2)*2
def get_Dist(i):
    fname = r'/data/new_trap/20200320/Bead1/Shaking/Shaking378/Shaking3_'+str(i)+'.h5'
    bb = BeadDataFile.BeadDataFile(fname=fname)    
    return 392.2 - aux.voltage_to_position(np.mean(bb.cant_pos[0]))

In [12]:
z_sep = np.mean(height_cal)
y_offset = np.mean(np.array(Parallel(n_jobs=10)(delayed(get_Yoffset)(i) for i in tqdm(range(0, 10000, 100)))))
stroke = np.mean(np.array(Parallel(n_jobs=10)(delayed(get_Stroke)(i) for i in tqdm(range(0, 10000, 100)))))
dist = np.mean(np.array(Parallel(n_jobs=10)(delayed(get_Dist)(i) for i in tqdm(range(0, 10000, 100)))))
G_lambda = 10
print(z_sep, y_offset, stroke, dist, G_lambda)

100%|██████████| 100/100 [00:00<00:00, 451.89it/s]


-15.859100393 4.88214369679 -202.765748131 13.9887662292 10


In [14]:
G_lambda = 20
test_shaking378 = force_vs_time(dist,z_sep,stroke,3,G_lambda,"z", 
                                yuk_or_grav="yuk",offset_y=y_offset, alpha=1e8, bead_size=7.6)
shaking378_template = np.array(test_shaking378[1])
test_shaking373 = force_vs_time(dist+5,z_sep,stroke,3,G_lambda,"z", 
                                yuk_or_grav="yuk",offset_y=y_offset, alpha=1e8, bead_size=7.6)
shaking373_template = np.array(test_shaking373[1])

Loading Gravity Data... Done!
68 2.04837466632e-05
Loaded Yukawa Force
Loading Gravity Data... Done!
68 2.04837466632e-05
Loaded Yukawa Force


In [18]:
def fft_norm(N, fsamp):
    "Factor to normalize FFT to ASD units"
    return np.sqrt(2 / (N * fsamp))
scale = np.sqrt(2)*fft_norm(5000, 5000)

_,ax = plt.subplots()
fft378 = np.abs(np.fft.rfft(shaking378_template))*scale
# fft378_2 = np.abs(np.fft.rfft(shaking378_template, norm='ortho'))*scale
angles378 = np.angle(np.fft.rfft(shaking378_template))
fft373 = np.abs(np.fft.rfft(shaking373_template))*scale
freq1 = np.fft.rfftfreq(len(shaking373_template), d=1./5000)
ax.semilogy(freq1, fft378*scale, label='378')
ax.semilogy(freq1, fft373*scale, label='373')
ax.set(xlim=(1,70), ylim=(1e-24,1e-19), xlabel='frequency [Hz]', ylabel='Force [N]', title=r'$\alpha=1x10^{8}$')
ax.legend()
print('Z force [N] at 36Hz. (378,373): (', fft378[freq1==36],',', fft373[freq1==36],')')
print('sensitivity [alpha]:', 1.25e-15/fft378[freq1==36]*1e8)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Z force [N] at 36Hz. (378,373): ( [  5.34105319e-18] , [  2.42016617e-18] )
sensitivity [alpha]: [  2.34036239e+10]


In [21]:
# 100 files of shaking data for nose floor
# data_bdfs = load_dir(dirname='/data/new_trap/20200320/Bead1/Shaking/Shaking378/',
#                          file_prefix='Shaking3_', start_file=0, max_file=1000)

# 100 files of shaking data for nose floor
data_bdfs = load_dir(dirname='/data/new_trap_processed/mockfiles/20200320/output/noise/non_blind_with_bkg/Test1/50/',
                         file_prefix='Noise_batch', start_file=0, max_file=100)
freq = data_bdfs[0].psd2('x')[0]

freq = data_bdfs[0].psd2('x')[0]
fftx4 = data_bdfs[0].psd2('x')[1]
ffty4 = data_bdfs[0].psd2('y')[1]
fftz4 = data_bdfs[0].psd2('z')[1]
for i in range(1,100):
    fftx4 += data_bdfs[i].psd2('x')[1]
    fftz4 += data_bdfs[i].psd2('z')[1]
    ffty4 += data_bdfs[i].psd2('y')[1]
fftx4/=100
ffty4/=100
fftz4/=100

1000  files in folder
100  files loaded


In [22]:
raw = data_bdfs[0].z2
for i in range(1,100):
    raw += data_bdfs[i].z2
raw = raw/100
fftz5 = np.abs(np.fft.rfft(raw))*scale

In [23]:
scale2 = np.sqrt(2)*fft_norm(50000, 5000)
1/gfw.scale_Z2, scale2

(5.4256575158892859e-18, 0.00012649110640673518)

In [24]:
# fftz3 = fftz1
_,ax = plt.subplots(1,2,figsize=(9.5,4))
# ax[0].semilogy(freq, np.sqrt(fftx_Wilson)*x_scale, label='Wilson 378-3')
# ax[0].semilogy(freq, np.sqrt(fftx_Wilson2)*x_scale, label='Wilson 378-3')
ax[0].legend()
# ax[1].semilogy(freq, np.sqrt(fftz4)/gfw.scale_Z2/np.sqrt(5000), label='Wilson 378-3')
ax[1].semilogy(freq, np.sqrt(fftz5)/gfw.scale_Z2, label='Wilson 378-3')
ax[1].semilogy(freq1, fft378*scale, label='378')
ax[1].legend() 
# ax[0].set(xlim=(1,70), ylim=(9e-18,1e-14), xlabel='Frequency [Hz.]', ylabel=r'ASD [N/$\sqrt{Hz}$]', title='X force')
_ = ax[1].set(xlim=(1,70), ylim=(2e-18,1e-14), xlabel='Frequency [Hz.]', title=r'Z force')
# print(np.sqrt(fftx_Wilson[freq==36])*x_scale/np.sqrt(10), np.sqrt(fftz_Wilson[freq==36])*z_scale/np.sqrt(10))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

No handles with labels found to put in legend.


In [1]:
def func_filter(i):
#     fname = r'/data/new_trap/20200320/Bead1/Shaking/Shaking378/Shaking3_'+str(i)+'.h5'
    fname = r'/data/new_trap/20200320/Bead1/Shaking/Shaking2_20200403/NoShaking_1_'+str(i)+'.h5'
#     fname = r'/data/new_trap/20200320/Bead1/Shaking/Shaking373/Shaking4_'+str(i)+'.h5'
#     fname = r'/data/new_trap_processed/mockfiles/20200320/output/noise/bkg_simple/0/0/Noise_batch_'+str(i)+'.h5'


    bb = BeadDataFile.BeadDataFile(fname=fname)
    res = bb.z2
    return (res)

In [2]:
del test_phases

NameError: name 'test_phases' is not defined

In [13]:
test_phases = np.array(Parallel(n_jobs=46)(delayed(func_filter)(i) for i in tqdm(range(1000))))

100%|██████████| 1000/1000 [00:03<00:00, 279.76it/s]


In [14]:
def func_sideband(x, gravity_freqs_all):
    res = []
    for freq_ in gravity_freqs_all:
        b, a = signal.butter(3, [2.*(freq_+1.5-bandwidth/2.)/5000, 2.*(freq_+1.5+bandwidth/2.)/5000], btype = 'bandpass')
        responsefilt = signal.filtfilt(b, a, x)
        res.append(np.std(responsefilt))
    return np.array(res)

In [15]:
ll = likelihood_analyser.LikelihoodAnalyser()

def calc_bdf3(i):

    data_z2 = np.mean(test_phases[i*100:(i+1)*100], axis=0)+sig_long
    
    fit_kwargs = {'A': 0, 'phi': 0, 'sigma':13.85,
              'error_A': 1, 'error_phi': 0.5, 'errordef': 1, 'error_sigma': 10,
              'limit_phi': [0, np.pi],
              'limit_A': [-100000, 100000], 'limit_sigma': [0, 1000000],
              'print_level': 0, 'fix_A': False, 'fix_phi': True, 'fix_sigma': False}
    
    noises_std = np.array([1])
#     noises_std = func_sideband(data_z2, gravity_freqs)
#     noises_std = noises_std/np.min(noises_std)

    mm_tmp = ll.find_mle_multiHarmoincs2(x=data_z2, template=shaking378_template, scales=gravity_scales, phases=gravity_phases, 
                                          signal_freqs=gravity_freqs, bandwidth=bandwidth, noises=noises_std,
                                          decimate=10, **fit_kwargs)
    print('MLEs: ', mm_tmp.values)
    PL_array = A_array*0
    if (mm_tmp.values[0]>A_array[0])&(mm_tmp.values[0]<A_array[-1]):
        PL_array = 2*10*ll.get_PL_multiHarmonics(A_array=A_array, **fit_kwargs)
        PL_array -= np.min(np.interp(np.arange(np.min(A_array),np.max(A_array),0.0001), A_array, PL_array))
        
    return PL_array

In [16]:
dist,z_sep,stroke,3,G_lambda,y_offset

(13.988766229192864,
 -15.85910039296383,
 -202.76574813093302,
 3,
 10,
 4.882143696792518)

In [18]:
gravity_scales, gravity_phases, gravity_freqs

(array([ -1.40895736e+17]), array([[ 1.84123774]]), array([36]))

In [17]:
## likelihood calculator stuff
gravity_freqs_all = 3*np.arange(1,13)
# gravity_freqs_all = np.array([36])
G_lambda = 10
test_shaking378 = force_vs_time(dist,z_sep,stroke,3,G_lambda,"z", 
                                yuk_or_grav="yuk",offset_y=y_offset, alpha=1e8, bead_size=7.6)
shaking378_template = np.array(test_shaking378[1])
A_array = np.arange(-15,10,0.1)

t_long = 10
fsamp = 5000
dt = 1. / fsamp
nsamp_long = int(t_long / dt)
time_long= np.arange(nsamp_long) * dt

fft_angles = np.angle(np.fft.rfft(shaking378_template))
fft_fft = np.abs(np.fft.rfft(shaking378_template)) * 2 / np.sqrt(5000 * 5000)
freq1 = np.fft.rfftfreq(len(shaking378_template), d=1./5000)
fname = r'/data/new_trap/20200320/Bead1/TransFunc/Repeat_TransFunc_20200104/TransFunc_Z_m300k_250s_1hz.h5'
tf_z = BeadDataFile.BeadDataFile(fname)
fft_angles2 = np.angle(np.fft.rfft(tf_z.z2))
freq2 = np.fft.rfftfreq(len(tf_z.z2), d=1./5000)
fft_angles3 = np.angle(np.fft.rfft(tf_z.electrode_data[0]))
freq3 = np.fft.rfftfreq(len(tf_z.electrode_data[0]), d=1./5000)

PLs = []

for drive_freq in gravity_freqs_all:
    phase_th = fft_angles[freq1==drive_freq]  ## theory 
    phase_th2 = fft_angles2[freq2==drive_freq] ## response phase
    phase_th3 = fft_angles3[freq3==drive_freq] ## drive phase
    phase_all = (phase_th+(phase_th2-phase_th3)+np.pi/2)%(2*np.pi)
    
    gravity_phases = np.array([phase_all])
    gravity_freqs = np.array([drive_freq])
    gravity_scales = np.array([gfw.scale_Z2])* np.interp(gravity_freqs, gfw.tf_freq, gfw.tf_ffts[2])
    
    
    amp = gravity_scales[0]*fft_fft[freq1==drive_freq][0]*0.0
    sig_long = amp * np.sin(2 * np.pi * drive_freq * time_long + phase_all)
    ll = likelihood_analyser.LikelihoodAnalyser()

    def calc_pl(i):

        data_z2 = np.mean(test_phases[i*500:(i+1)*500], axis=0)

        fit_kwargs = {'A': 0, 'phi': 0, 'sigma':13.85,
                  'error_A': 1, 'error_phi': 0.5, 'errordef': 1, 'error_sigma': 10,
                  'limit_phi': [0, np.pi],
                  'limit_A': [-100000, 100000], 'limit_sigma': [0, 1000000],
                  'print_level': 0, 'fix_A': False, 'fix_phi': True, 'fix_sigma': False}

        noises_std = np.array([1])

        mm_tmp = ll.find_mle_multiHarmoincs2(x=data_z2, template=shaking378_template, scales=gravity_scales, phases=gravity_phases, 
                                              signal_freqs=gravity_freqs, bandwidth=bandwidth, noises=noises_std,
                                              decimate=10, **fit_kwargs)
        print('MLEs: ', mm_tmp.values)
        PL_array = A_array*0
#         if (mm_tmp.values[0]>A_array[0])&(mm_tmp.values[0]<A_array[-1]):
        PL_array = 2*10*ll.get_PL_multiHarmonics(A_array=A_array, **fit_kwargs)
#             PL_array -= np.min(np.interp(np.arange(np.min(A_array),np.max(A_array),0.0001), A_array, PL_array))

        return PL_array

    PLs.append(np.array(Parallel(n_jobs=40)(delayed(calc_pl)(i) for i in tqdm(range(2)))))

Loading Gravity Data... 

  0%|          | 0/2 [00:00<?, ?it/s]

Done!
81 9.74070978211e-06
Loaded Yukawa Force
No height data
No cantilever data
Error loading spinning data
No laser power data


100%|██████████| 2/2 [00:00<00:00, 1774.62it/s]


### Do the econstruction on harmonic-by-harmonic basis

In [115]:
import copy

PLscopy = copy.deepcopy(PLs)

PLs_all = []
for PLs_ in PLscopy:
    PLs5_tmp = PLs_[0]
    for pl in PLs_[1:]:
        PLs5_tmp += pl
    PLs5_tmp -= np.min(np.interp(np.arange(np.min(A_array),np.max(A_array),0.0001), A_array, PLs5_tmp))
    PLs_all.append(PLs5_tmp)

In [116]:
fig ,ax = plt.subplots(1,4,figsize=(9,2.5), sharey=True)
_ = [a.set(ylim=(0,6), xlim=(np.min(A_array),np.max(A_array)), xlabel=r'$\alpha_{fit} \, [10^{8}]$') for a in ax[1:]]
_ = ax[0].set(ylim=(0,6), xlim=(np.min(A_array),np.max(A_array)), xlabel=r'$\alpha_{fit} \, [10^{8}]$', ylabel = 'Profile-Likelihood')
for i,harmonics in enumerate([4 ,6, 10, 11]):
    ax[i].plot(A_array, PLs_all[harmonics-1], label=str(harmonics*3)+' Hz.-shaking')
#     ax[i].plot(A_array, PLs_conservative[harmonics-1], label=str(harmonics*3)+' Hz.-shaking')
    ax[i].plot(A_array, A_array*0 + 2.71, 'r--')
_ = [a.legend() for a in ax]
fig.suptitle(r'PL vs. $\alpha_{fit} \, [10^{8}]$', fontsize=10)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 0.98, 'PL vs. $\\alpha_{fit} \\, [10^{8}]$')

In [110]:
all_mles = []
for i,pl in enumerate(PLs_all):
    min_alpha = A_array[np.argmin(pl)]
    all_mles.append(min_alpha)
    print(gravity_freqs_all[i], min_alpha)
all_mles = np.array(all_mles)

3 0.2
6 -3.6
9 -8.3
12 -13.1
15 -5.3
18 -5.8
21 -4.2
24 -5.9
27 -6.1
30 -4.3
33 -4.4
36 -4.8


In [60]:
mle_lim = np.array([gravity_freqs_all, all_mles])
np.savetxt('bkg_simple2_mle_NP.csv',mle_lim,delimiter=',')

In [46]:
mle_lim

array([[  3. ,   6. ,   9. ,  12. ,  15. ,  18. ,  21. ,  24. ,  27. ,
         30. ,  33. ,  36. ],
       [ -3.6,  -3.9,  -8.4,  -4.7,  -5.6,   2.9,  -4.2,  -5.6,  -5.3,
         -4.5,  -4.2,  -4.9]])

In [65]:
!pwd

/home/analysis_user/New_trap_code/Scripts/Gravity paper


In [61]:
## cnostruction for each harmonic separately 
PLs_conservative = []
PLscopy = copy.deepcopy(PLs_all)
for pl in PLscopy:
    min_alpha = A_array[np.argmin(pl)]
    min_value = pl[np.argmin(pl)]
    if (min_alpha>(np.min(A_array)-0.2))&(min_alpha<(np.max(A_array)+0.2)):
        if (min_alpha>-0.001):
            for i,A_ in enumerate(A_array):
                if A_<min_alpha:
                    pl[i] = min_value
            PLs_conservative.append(pl)
        else:
            for i,A_ in enumerate(A_array):
                if A_>min_alpha:
                    pl[i] = min_value
            PLs_conservative.append(pl)

In [62]:
## sum all harmonics after construction
PLs_summed_harmonics = []
PLscopy = copy.deepcopy(PLs_conservative)
PLs_summed_harmonics = PLscopy[0]
for pp_ in PLscopy[1:]:
    PLs_summed_harmonics += pp_

In [63]:
## sum all harmonics before construction
PLs_summed_harmonics2 = []
PLscopy = copy.deepcopy(PLs_all)
PLs_summed_harmonics2 = PLscopy[0]
for pp_ in PLscopy[1:]:
    PLs_summed_harmonics2 += pp_

In [64]:
_,ax = plt.subplots()

PLs_summed_harmonics -= np.min(np.interp(np.arange(np.min(A_array),np.max(A_array),0.001), A_array, PLs_summed_harmonics))
PLs_summed_harmonics2 -= np.min(np.interp(np.arange(np.min(A_array),np.max(A_array),0.001), A_array, PLs_summed_harmonics2))
ax.plot(A_array, PLs_summed_harmonics, '*-', label='summing harmonics after the construction')
ax.plot(A_array, PLs_summed_harmonics2, '*-', label='summing harmonics before the construction')
ax.legend()
ax.plot(A_array, A_array*0 + 2.71, 'r--')
ax.set(ylim=(0,10), xlim=(np.min(A_array),np.max(A_array)), xlabel=r'$\alpha_{signal} \, [10^{8}]$', ylabel = 'Profile-Likelihood')
ax.set(title=r'$\alpha_{true}=1\times10^7$')
print(np.interp(7, PLs_summed_harmonics[100:], A_array[100:]))
print(np.interp(2.7, PLs_summed_harmonics[100:], A_array[100:]))
# print(np.interp(2.71, PLs_summed_harmonics[100:0:-1], A_array[100:0:-1]))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

6.85072697002
6.16322486781


In [197]:
0.18/5

0.036

In [169]:
PLscopy.shape

(12, 1000, 200)

In [173]:
t = []
PLscopy = copy.deepcopy(PLs_noise_alpha5)
PLscopy = np.array(PLscopy)
for i in range(1000):
    pltmp = PLscopy[1][i]
    for j in range(2,12):
        pltmp += PLscopy[j][i]
    min_value = pltmp[np.argmin(pltmp)]
    print(np.interp(5, A_array, pltmp)-min_value)
    t.append(np.interp(5, A_array, pltmp)-min_value)

0.0481073390993
1.03919301901
0.0
1.26141204827
0.133708643619
0.534099541389
0.0024048897817
1.87988524427
1.67766416236
0.022661323032
0.847144974742
0.0133318647754
1.29938139732
0.813677643083
4.1697061608
0.0
0.00808518480335
0.744674460746
1.22571643455
0.00949366444956
0.0196655983286
0.00941215570265
0.418892107983
0.515835195979
0.370115254942
0.0237526287583
0.0343808737334
0.024444276643
0.036135337471
0.0167110229286
0.499259447171
0.540693560038
0.194532171893
0.46051489628
3.31417394038
0.00508113774777
0.030592014386
0.00182865433362
0.351631784822
0.462761276699
0.0481254279989
0.0147722545548
1.53749104478
1.02094527375
0.147069347467
0.0
0.51384750409
0.000126758860461
0.0103955532259
0.436976836165
0.0142620321481
0.492876965266
0.00530699379851
0.379561994795
2.04291354649
0.517450732953
0.401708333837
0.187176395342
0.606352604987
0.00466518787016
0.00881737647433
0.583144429444
0.663399519811
0.812344158885
0.165993378319
1.23111870569
0.933824168616
0.20860043856

In [182]:
PLscopy.shape

(12, 1000, 200)

In [185]:
t2 = []
PLscopy = copy.deepcopy(PLs_noise_alpha5)
PLscopy = np.array(PLscopy)
for i in range(1000):
    for j in range(1,12):
        min_alpha = A_array[np.argmin(PLscopy[j][i])]
        min_value = PLscopy[j][i][np.argmin(PLscopy[j][i])]
        for k,A_ in enumerate(A_array):
            if A_<min_alpha:
                PLscopy[j][i][k] = min_value
        
    pltmp = PLscopy[1][i]
    for j in range(2,12):
        pltmp += PLscopy[j][i]
    min_value = pltmp[np.argmin(pltmp)]
    print(np.interp(5, A_array, pltmp)-min_value)
    t2.append(np.interp(5, A_array, pltmp)-min_value)

2.76041625942
4.77230541504
1.39514123292
1.58489164839
3.95558136219
4.44430945154
4.95949291625
8.99270558717
4.52866940419
1.41661299717
5.27549955543
2.51243662195
2.00734442661
3.17829119308
8.57430875312
2.79325285448
3.70719276672
0.150364505956
2.39265639079
1.21900981057
2.60377745604
4.75692105431
1.99130429828
3.58160675562
3.12572429589
2.53383351063
4.0842010578
1.68529883341
4.08504313257
2.40019324483
2.13967140619
1.84342465622
1.40208244369
1.05019505926
6.98248433623
3.05271700717
5.15578524164
3.20428839621
0.754682063232
2.13898847742
1.4142396404
1.65978000587
2.50699398341
0.787675775973
4.16601607433
2.32134767921
2.74005761034
2.01530198197
2.80972746314
0.840648161285
3.24718517393
0.731490175932
3.02568103601
4.10984112983
6.7466161427
1.50825165126
4.55251721498
4.92018831351
0.537068357387
4.30892911664
2.04576782759
3.30669517295
3.0962442579
4.61222335574
2.36931331484
0.0587341061259
3.44120623141
3.37668555047
2.17408915774
2.53365409044
2.21938336681
7.

2.44054912061
3.92500168142
0.849235662433
3.42594614296
2.4202661809
1.66511844768
3.08385740885
3.60856723827
3.24999839608
3.47420922052
1.1640092767
5.0512043023
2.71467870437
6.03066786834
3.39091315464
4.96390354522
6.14018616275
3.27884206236
5.79669824758
0.651586417312
1.49879777344
3.7838646956
2.13191425657
1.54924955122
1.32279237988
2.30448897716
2.93364747871
1.15892156561
5.37076907355
3.22283931062
5.14701972102
4.13379307946
3.26530285969
6.22130558815
0.0390054224158
2.87722514348
7.01976857576
2.68815150435
1.78900617712
2.71224218785
3.53423329325
3.66652565869
1.04203774349
6.87921813607
3.21302562148
1.91241837632
1.6249723233
4.78611458364
1.53062321065
3.4511376665
2.84760312294
3.21850548811
1.21561359644
5.12325255374
7.13238999655
3.08047907348
4.09129724857
2.02845955206
3.44825424058
0.74895697737
2.84107585899
4.61977606448
0.824224078468
3.28223907489
1.40190620145
6.20425813277
0.998022744254
4.85039670081
2.75412395074
2.15380170163
5.49356057555
5.2019

In [ ]:
t3 = []
PLscopy = copy.deepcopy(PLs_noise_alpha5)
PLscopy = np.array(PLscopy)
for i in range(1000):
    for j in range(1,12):
        min_alpha = A_array[np.argmin(PLscopy[j][i])]
        min_value = PLscopy[j][i][np.argmin(PLscopy[j][i])]
        for k,A_ in enumerate(A_array):
            if A_<min_alpha:
                PLscopy[j][i][k] = min_value
        
    pltmp = PLscopy[1][i]
    for j in range(2,12):
        pltmp += PLscopy[j][i]
    min_value = pltmp[np.argmin(pltmp)]
    print(np.interp(5, A_array, pltmp)-min_value)
    t3.append(np.interp(5, A_array, pltmp)-min_value)

In [210]:
import scipy.stats as stats

_,ax = plt.subplots()
ax.hist(t, bins=50, range=(0,6), alpha=0.4, log=True, label='multi-harmonic')
ax.hist(np.array(t2), bins=50, range=(0,6), alpha=0.4, log=True, label='harmonic-by-harmonic')
x = np.arange(0, 6, .05)
plt.plot(x, stats.chi2.pdf(x, df=1)*1000/50*6, color='r', lw=2, label='scaled chi square')
ax.legend()
ax.set(xlabel=r'-2*log($\lambda (\alpha_0 = 5)$)', title=r'f($q_{\alpha_0}|\alpha_0$)')
print(sum(np.array(t)>2.71)/1000, sum(np.array(t2)>7)/1000)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

0.036 0.046


In [302]:
_,ax = plt.subplots(1,2,figsize=(9.2,4))
xt_freq, xt_zx = tf_x.psd2('z') 
ax[0].semilogy(xt_freq, xt_zx)
ax[0].set(xlim=(0,100))
xt_freq, xt_zz = tf_z.psd2('z') 
ax[0].semilogy(xt_freq, xt_zz, alpha=0.5)
ax[0].set(xlim=(0,100))
xt_freqs = 3*np.arange(1,30)
xt_percent = [np.sqrt(xt_zx[xt_freq==freq_]/xt_zz[xt_freq==freq_]) for freq_ in xt_freqs]
ax[1].scatter(xt_freqs, xt_percent)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [304]:
i=100
drive_freq=12
bandwidth = 1
fsamp=5000
b, a = signal.butter(3, [2.*(drive_freq-bandwidth/2.)/fsamp, 2.*(drive_freq+bandwidth/2.)/fsamp ], btype = 'bandpass')
filtered_z = signal.filtfilt(b, a, test_phases[i])
filtered_x = signal.filtfilt(b, a, test_phasesx[i])

tf_filtered_z = signal.filtfilt(b, a, tf_x.z2)
tf_filtered_x = signal.filtfilt(b, a, tf_x.x2)

np.std(filtered_z)/np.std(filtered_x*500000), np.std(tf_filtered_z)/np.std(tf_filtered_x*500000)

(5.7622459800570498, 2.5730988976452327)

In [276]:
_,ax = plt.subplots(figsize=(9.2,4))
ax.scatter(range(50000), filtered_x*500000)
ax.scatter(range(50000), filtered_z, alpha=0.5)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(0.37405867940629484, 0.26324026963027719)

In [274]:
_,ax = plt.subplots(figsize=(9.2,4))
ax.scatter(range(50000), tf_filtered_x[:50000]*500000)
ax.scatter(range(50000), tf_filtered_z[:50000], alpha=0.5)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …